# TP Relation Extraction

## Brieuc Calvez - Simon Ardiet (3A IF)

## Introduction

Dans ce TP nous allons:

1. Découvrir la tache d’extraction de relation par réseaux neuronaux  (« neural relation extraction », NRE)
2.  Comprendre le fonctionnement d’un modèle d’extraction de relation avec un encodeur BERT
3. Découvrir la tache de reconnaissance d'entités nommées (« named-entity recognition », NER)
4. Coder une pipeline d’extraction de relation par réseaux neuronaux pour des jeux de données textuels

Avec les outils suivants:
1. La libraire [OpenNRE](https://github.com/thunlp/OpenNRE), basée sur Pytorch et HuggingFace’s Transformers, pour la tache d’extraction de relation par réseaux neuronaux
2. [HuggingFace’s Transformers](https://huggingface.co/transformers/) : une bibliothèque basée sur Pytorch pour le traitement automatique des langues et notamment les modèles neuronaux de type Transformer (comme BERT)
3. Google Colab, qui héberge ce *Jupyter Notebook*. Avant de commencer le TP, vous pouvez consulter des pages d'introductions [à Colab](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb#scrollTo=YHI3vyhv5p85) et [aux Notebooks](https://realpython.com/jupyter-notebook-introduction/)


Contrairement au dernier TP, cette fois-ci nous n’aurons pas besoin d’utiliser une GPU car nous n’allons pas entraîner des nouveaux modèles ou faire de l’inférence sur des grands jeux de données: une CPU suffira. Nous pouvons vérifier si l’on est en train d’utiliser une CPU ou une GPU avec les lignes suivantes:

In [148]:
import torch

if torch.cuda.is_available():
  print("GPU is available.")
  device = torch.cuda.current_device()
else:
  print("Will work on CPU.")

GPU is available.


Nous avons besoin d’installer l’outil OpenNRE. Pour éviter de devoir ré-télécharger le répertoire  GitHub de OpenNRE à chaque fois qu’on ré-initialise le fichier Colab, il est convenant de monter notre répertoire Google Drive et y télécharger le répertoire OpenNRE de façon à l’avoir toujours disponible. Ainsi, à chaque fois que nous allons ré-initialiser le fichier Colab, il nous suffira de monter notre Goodle Drive pour avoir accès à la libraire OpenNRE.

> ATTENTION: modifiez *tp_path_in_drive* pour pointer vers le repertoire où vous avez placé le fichier tp_re.ipynb, vous allez télécharger OpenNRE dans le meme réépertoire. Si vous êtes sur votre machine locale, vous n'avez pas besoin de monter le Drive, mais juste de faire le clonage du réépertoire OpenNRE.

In [149]:
import os
from google.colab import drive

tp_path_in_drive = '/content/gdrive/MyDrive/Ensimag/3A Ensimag/Traitement du langage/4-tp_re'
opennre_path_in_drive = tp_path_in_drive + '/OpenNRE'

# mount Google Drive
drive.mount('/content/gdrive')

if not os.path.isdir(opennre_path_in_drive):
  # OpenNRE is not already present in Google Drive
  if not os.path.isdir(tp_path_in_drive):
    # make directory for the TP if necessary
    os.makedirs(tp_path_in_drive, exist_ok=True)
  # change directory to the TP directory
  os.chdir(tp_path_in_drive)
  # clone OpenNRE repo
  print("Cloning repo...")
  os.system('git clone https://github.com/thunlp/OpenNRE.git')
  print("...done!")
else:
  print("OpenNRE is already present in Google Drive under {0}".format(opennre_path_in_drive))

# Change current dir to OpenNRE
os.chdir(opennre_path_in_drive)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
OpenNRE is already present in Google Drive under /content/gdrive/MyDrive/Ensimag/3A Ensimag/Traitement du langage/4-tp_re/OpenNRE


In [150]:
# Update requirements
!sed -i '/transformers==3.0.2/c\transformers==3.4.0' requirements.txt

Nous pouvons désormais continuer avec l’installation.

In [151]:
!pip install -r requirements.txt
!python setup.py install

running install
running bdist_egg
running egg_info
writing opennre.egg-info/PKG-INFO
writing dependency_links to opennre.egg-info/dependency_links.txt
writing top-level names to opennre.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'opennre.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/opennre
copying build/lib/opennre/__init__.py -> build/bdist.linux-x86_64/egg/opennre
copying build/lib/opennre/pretrain.py -> build/bdist.linux-x86_64/egg/opennre
creating build/bdist.linux-x86_64/egg/opennre/encoder
copying build/lib/opennre/encoder/__init__.py -> build/bdist.linux-x86_64/egg/opennre/encoder
copying build/lib/opennre/encoder/base_encoder.py -> build/bdist.linux-x86_64/egg/opennre/encoder
copying build/lib/opennre/encoder/bert_encoder.py -> build/bdist.linux-x86_64/egg/opennre/encoder
copying build/lib/opennre/encod

## Les modéles pour l'extraction de relation

À part offrir un cadre pour l’implémentation et l’entraînement de modelés pour l’extraction de relation, OpenNRE offre aussi des modèles déjà entraînés sur différents jeux de données, et donc capables de détecter différents types de relations entre entités. 

Ici, nous allons employer un modèle entraîné sur Wiki80 (dataset introduit par [le papier OpenNRE](https://www.aclweb.org/anthology/D19-3029/)), un jeu de données contenant des phrases collectées sur Wikipedia et Wikidata, ainsi que des rélations entre leurs entités. Si vous voulez en savoir plus sur Wiki80, vous pouvez le télécharger avec le script [download_wiki80.sh](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/benchmark/download_wiki80.sh) 

In [152]:
import opennre
model = opennre.get_model('wiki80_bert_softmax')

2022-03-14 10:56:48,670 - root - INFO - Loading BERT pre-trained checkpoint.


Nous pouvons utiliser ce modèle pour calculer la relation entre un mot «tête» et un mot «queue» qui sont contenus dans un texte. Il suffit de passer au modèle le texte ainsi que la position de la tête et de la queue. Le modèle retournera la relation de la queue à l’égard de la tête, ainsi que la probabilité qu’il associe à cette rélation. Par example, nous pouvons inféérer la relation entre **Áed Uaridnach** et **Máel Dúin mac Máele Fithrich** de la façon suivante:

In [153]:
item = {'text': 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', 'h': {'pos': (78, 91)}, 't': {'pos': (18, 46)}}
print(model.infer(item))

('child', 0.9812852144241333)


#### Exercice 1



Écrire une fonction `to_input_format(text, head, tail)` qui nous permet de trouver la position de deux mots (une tête et une queue) dans un texte, et qui retourne un dictionnaire contenant le texte et les deux positions suivant le format requis par la fonction `model.infer()`.

In [154]:
import re

def to_input_format(text, head, tail):
  """
  Args:
    text: a string of text
    head: a string of text representing a word contained in text
    tail: a string of text representing a word contained in text but different from head
  Returns:
    A dictionary containing the text and the position of the head and tail within it,
    following the input format required by an OpenNRE model (see exemple above).
  """
  pos_tail = (text.find(tail), text.find(tail) + len(tail))
  pos_head = (text.find(head), text.find(head) + len(head))
  return {'text' : text, 'h' : {'pos': pos_head}, 't' : {'pos': pos_tail}}

In [155]:
# Test your code with this snippet
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
head = 'Áed Uaridnach'
tail = 'Máel Dúin mac Máele Fithrich'
test_item = to_input_format(text, head, tail)
try:
  assert model.infer(item) == model.infer(test_item)
  print("Good job!")
except AssertionError:
  print("Something is wrong with your function, try again!")

Good job!


#### Exercice 2

Décrire l’architecture du modèle qu’on vient de télécharger, la logique et le fonctionnement de chacun de ses composants:

1. Sentence-encoder: pourquoi l'on utilise un encodeur type BERT ?
2. À quoi sert la couche BertPooler ?
3. À quoi sert la couche linéaire finale (fc) ? pourquoi elle réduit à 80 la dimension du vecteur sortant de l’encodeur?
4. À quoi sert la fonction Softmax ?
5. À quoi sert la fonction de dropout que l'on appliuque à la sortie du réseau, ainsi que dans chaque couche de BERT ?

Aide: vous pouvez inspecter un modèle Pytorch avec `print(model)`. Pour mieux le comprendre, vous pouvez aussi voir [son code](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/model/softmax_nn.py#L5), [la déscription du modèle BERT de Huggingface](https://huggingface.co/docs/transformers/v4.16.2/en/model_doc/bert#transformers.BertModel) ainsi que [son code](https://github.com/huggingface/transformers/blob/v4.16.2/src/transformers/models/bert/modeling_bert.py#L848).

In [156]:
print(model)

SoftmaxNN(
  (sentence_encoder): BERTEncoder(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
        

**Q1** : BERT repousse les limites du traitement du langage naturel en combinant deux technologies :
- Il est basé sur un réseau profond de "Transformer encoders", un type de réseau qui peut traiter efficacement de longs textes.
- Il est bidirectionnel, ce qui signifie qu'il utilise l'ensemble du passage du texte pour comprendre le sens de chaque mot.

**Q2** : La "pooler layer" prend la représention en sortie correspondant au premier "token", et l'utilise pour certaines tâches en aval. La "pooler layer", applique une transformation linéaire sur la représentation du premier "token". La transformation linéaire est entraînée en utilisant une stratégie NSP.
Voici le code correspondant à cette pooler layer : 

```
(pooler): BertPooler(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (activation): Tanh()
      )
```

**Q3** : Voici le code correspondant à cette layer FC : 

```
(fc): Linear(in_features=768, out_features=80, bias=True)
  (softmax): Softmax(dim=-1)
  (drop): Dropout(p=0.5, inplace=False)
```

Dans un réseau de neurones, une couche «fully connected» se trouver généralement dans les dernières couches et permet de compiler les données extraites des autres couches, et de mettre en forme la sortie finale. C’est une couche extrêmement coûteuse puisque toutes les entrées d’une couche sont connectées à toutes les fonctions d’activation d’une autre couche.

Ici, on remarque via le paramètre `out_features=80` que le vecteur de sortie a une dimension de 80 afin d’être conforme au dataset Wiki80, qui est composé de 80 relations. 

**Conforme dans quel sense? C'est quoi la fonction de cette couce FC? --> Cette couche calcule la probabilité associée à chaque classe (chaque rélation définie dans le dataser Wiki80) à partir de l'information contenue dans la représentation véctorielle du token [CLS].**

**Q4** : La fonction `Softmax` est utilisée pour normaliser la sortie des réseaux neuronaux afin qu'elle soit comprise entre 0 et 1, et dont la somme totale est 1. Ceci permet de représenter la "probabilité" de certitude dans la sortie du réseau. **Il est utile d'utiliser une representation entre 0 et 1 (au lieu de [-inf,+inf] pour le calcul de la fonction de perte)**

**Q5** : Le dropout est l’une des nombreuses techniques de pénalité utilisée afin de réduire l’overfitting (i.e.  l’apprentissage de bruit). On applique aux neurones une probabilité qu’ils soient désactivés à chaque epoch. A chaque passe, le modèle apprendra avec une configuration de neurones aléatoire et différente. Cela viendra «pertuber» les caractéristiques apprises par le modèle. **Ce n'est pas très claire dans votre explication comment le dropout aide le réseau à ne pas overfitter.**

## Encodeur

#### Exercice 3

Maintenant, nous allons essayer de mieux comprendre le fonctionnement de ce modèle avec un focus sur son encodeur, qui est définit dans le fichier [bert_encoder.py](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/encoder/bert_encoder.py#L7). Ce qui est spécifique à la tache de NRE dans ce modèle n’est pas l’architecture, mais plutôt la façon dont la séquence en input est tokenisée. Avec l’objectif de bien comprendre comment cet encodeur gère son input, répondez en détail aux questions suivantes qui se référent à la méthode `tokenize`. Si vous utilisez du code pour vous aider à répondre (conseillé), veuillez le joindre à vos réponses textuelles.

1. Qu’est-ce qu’elle sont les variables `sent0`, `ent0`, `sent1`, `ent1`, `sent2` ?
2. Qu’est-ce que c’est `re_tokens` ?
3. Dans le _forward_, le `BERTEncoder` prend en entrée exclusivement la séquence textuelle qui a étée préalablement tokenisée. Comment est-il capable de distinguer la tête et la queue en sort de (apprendre à) prédire la relation entre les deux ?
4. Qu’est-ce que c’est le « Padding » et pourquoi est-il utile ?
5. Qu’est-ce que c’est l’ « Attention mask » et pourquoi est-elle utile ? [Aide ici](https://huggingface.co/docs/transformers/v4.16.2/en/glossary#attention-mask)
6. **[BONUS]** Quelle est la différence entre la classe `BERTEncoder` et la classe `BERTEntityEncoder`, définie dans le même fichier ?


Aide : vous pouvez accéder à la méthode `tokenize` pour la tester avec `model.sentence_encoder.tokenize(item)`

**Réponse aux questions :**
 
1. Ces variables sont des sous-ensembles de la phrase globale `sentence` qui vont être tokenizés. On va ensuite les “additioner” entre eux de sorte à ce que l’on repère *via* un token `['[unused0]']` les deux parties de la phrase à analyser.
 
2. `re_tokens` représente la phrase entièrement tokenizée, avec les balises `[CLS]` et `[SEP]` que l'on a vu lors du dernier TP.

3. La tête et la queue sont distingués *via* leur position. Ce sont les variables `pos1` et `pos2`* **Dans a question il est specifié ue BERT ne prend pas en entrée pos1 ni pos2. Ainsi, le réseau BERTEncoder apprend à distinguer la tête et la queue du reste du texte grace aux tokens "unused" qui ont été introduits à la tokenization**

4. Habituellement, la longueur maximale d'une phrase dépend des données sur lesquelles nous travaillons. Pour les phrases qui sont plus courtes que cette longueur maximale, nous devrons ajouter des paddings (tokens vides) aux phrases pour compenser la longueur. Dans l'implémentation originale, le token `[PAD]` est utilisé pour représenter les paddings de la phrase. **Pourquoi faut-il compenser la longueur? -> pour qu'on puisse representer un lot de phrases avec une matrice, ce qui est nécessaire au réseau pour traiter les phrases en parallèle.**

5. L'«Attention mask» indique au modèle les tokens qui doivent être pris en charge et ceux qui ne doivent pas l'être. **Quels tokens ne sont pas pris en charge?**

In [157]:
# Nous n'avons pas utilisé de code pour répondre aux questions, la lecture
# de la documentation a été suffisante.

## NRE Pipeline

Nous allons maintenant programmer une application qui prend en entrée une phrase et donne en sortie deux entités nommées dans la phrase ainsi que la relation entre eux.

Pour ce faire, nous avons besoin :

1. d’un système NER, qui reconnaît les entités nommées dans la phrase
2. d’un système NRE, comme celui que nous avons utilisé jusqu’à là

#### Exercice 4

En vous aident avec la documentation de HuggingFace, instanciez une [pipeline](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipeline#the-pipeline-abstraction) `ner_pipeline` pour la reconnaissance d'entités nommées, avec le modèle et le tokeniseur pré-entraînes [dbmdz/bert-large-cased-finetuned-conll03-english](https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).

Vous pouvez tester la bonne réussite de l’exercice avec le code ci-dessous.

**Indice** : la solution consiste en deux lignes de code : l'une pour importer la classe pipeline, l'autre pour instancier la bonne pipeline.

In [158]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model_ner = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

In [159]:
ner_pipeline = pipeline("ner", model=model_ner, tokenizer=tokenizer, device=device)

In [160]:
# Apply your ner_pipeline to some sentences to see how it works,
# Then you can test your code with this snippet
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
try:
  assert ner_pipeline(text) == [
                                {'entity': 'I-PER', 'index': 6, 'score': 0.982085645198822, 'word': 'M'},
                                {'entity': 'I-PER', 'index': 7, 'score': 0.8588601350784302, 'word': '##á'},
                                {'entity': 'I-PER', 'index': 8, 'score': 0.8766007423400879, 'word': '##el'},
                                {'entity': 'I-PER', 'index': 9, 'score': 0.8375428915023804, 'word': 'D'},
                                {'entity': 'I-PER', 'index': 10, 'score': 0.4897182583808899, 'word': '##ú'},
                                {'entity': 'I-PER', 'index': 11, 'score': 0.901253342628479, 'word': '##in'},
                                {'entity': 'I-PER', 'index': 12, 'score': 0.5821399688720703, 'word': 'mac'},
                                {'entity': 'I-PER', 'index': 13, 'score': 0.8884768486022949, 'word': 'M'},
                                {'entity': 'I-PER', 'index': 14, 'score': 0.7004077434539795, 'word': '##á'},
                                {'entity': 'I-PER', 'index': 15, 'score': 0.8791088461875916, 'word': '##ele'},
                                {'entity': 'I-PER', 'index': 16, 'score': 0.9720047116279602, 'word': 'Fi'},
                                {'entity': 'I-PER', 'index': 17, 'score': 0.697806715965271, 'word': '##th'},
                                {'entity': 'I-PER', 'index': 18, 'score': 0.697088360786438, 'word': '##rich'},
                                {'entity': 'I-PER', 'index': 26, 'score': 0.9203088283538818, 'word': 'Á'},
                                {'entity': 'I-PER', 'index': 27, 'score': 0.9416706562042236, 'word': '##ed'},
                                {'entity': 'I-PER', 'index': 28, 'score': 0.9702795147895813, 'word': 'U'},
                                {'entity': 'I-PER', 'index': 29, 'score': 0.8428962230682373, 'word': '##ari'},
                                {'entity': 'I-PER', 'index': 31, 'score': 0.7077912092208862, 'word': '##ach'}
                              ]
  print("Good job!")
except AssertionError:
  print("Something might be wrong with your pipeline.")
  print("Check the cell down below to compare, but this is normal.")

Something might be wrong with your pipeline.
Check the cell down below to compare, but this is normal.


In [161]:
ner_pipeline(text)

[{'entity': 'I-PER', 'index': 6, 'score': 0.9820857644081116, 'word': 'M'},
 {'entity': 'I-PER', 'index': 7, 'score': 0.8588611483573914, 'word': '##á'},
 {'entity': 'I-PER', 'index': 8, 'score': 0.8766009211540222, 'word': '##el'},
 {'entity': 'I-PER', 'index': 9, 'score': 0.8375439047813416, 'word': 'D'},
 {'entity': 'I-PER', 'index': 10, 'score': 0.489718496799469, 'word': '##ú'},
 {'entity': 'I-PER', 'index': 11, 'score': 0.9012538194656372, 'word': '##in'},
 {'entity': 'I-PER', 'index': 12, 'score': 0.5821399688720703, 'word': 'mac'},
 {'entity': 'I-PER', 'index': 13, 'score': 0.8884763717651367, 'word': 'M'},
 {'entity': 'I-PER', 'index': 14, 'score': 0.7004097104072571, 'word': '##á'},
 {'entity': 'I-PER',
  'index': 15,
  'score': 0.8791093826293945,
  'word': '##ele'},
 {'entity': 'I-PER', 'index': 16, 'score': 0.9720048308372498, 'word': 'Fi'},
 {'entity': 'I-PER', 'index': 17, 'score': 0.6978076100349426, 'word': '##th'},
 {'entity': 'I-PER',
  'index': 18,
  'score': 0.6970

#### Exercice 5


Nous pouvons finalement développer une pipeline NRE reposante sur notre modèle de NRE et notre pipeline NER. 

Écrivez ci-dessous une classe `NREPipeline` équipée (entre autres) d'une méthode `__call__(self, text)` qui prend un texte en entrée et effectue les opérations suivantes :

- elle reconnaît les entités dans le texte
    - retenir seulement les deux entités auxquelles la pipeline NER associe la probabilité la plus élevée, écarter les autres (si presentes)
    - si la pipeline NER ne reconnaît qu’une seule entités dans le texte, `__call__(self, text)` retourne None (voir le test en bas) car il n’y a aucune relation à prédire
- elle donne en sortie une liste en format `[e1, e2, rel, p]` où :
    - `e1` est la première entité reconnue dans le texte (entre les deux plus probables, la première qui apparaître dans le texte)
    - `e2` est la deuxième entité  reconnue dans le texte (entre les deux plus probables, la deuxième qui apparaître dans le texte)
    - `rel` est la relation qu’il y a entre `e1` et `e2`
    - `p` est la probabilité associée à la relation `rel` par le modèle de NRE

Pour vérifier la bonne qualité de votre classe `NREPipeline`, utilisez l’extrait de code ci-dessous. Le résultat devrait ressabler à celui-ci :

````
Sentence 0: He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).
System out:  ['Máel Dúin mac Máele Fithrich', 'Áed Uari', 'father', 0.9923498034477234]
------------------------------
Sentence 1: He was the son of Máel Dúin
System out:  None
------------------------------
Sentence 2: Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .
System out:  ['I', 'World Heritage Site', 'heritage designation', 0.9991846680641174]
------------------------------
Sentence 3: It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .
System out:  None
------------------------------
Sentence 4: Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church .
System out:  ['Earleville', "St . Stephen ' s Episcopal Church", 'location', 0.9127373099327087]
------------------------------
````

In [162]:
# Write your pipeline in this cell. The fun begins here, because we are coding a whole Python class from scratch !

class NREPipeline(object):
  def __init__(self, ner_pipeline, nre_model):
    self.ner_pipeline = ner_pipeline
    self.ner_pipeline.grouped_entities=True
    self.nre_model = nre_model

  def __call__(self, text):
   allScores = ner_pipeline(text)
   if len(allScores) <= 1:
     return None
   # on supprime tous les éléments qui contiennent exactement le token [UNK]
   i = 0
   while i < len(allScores):
     if allScores[i]['word'] == "[UNK]":
       allScores.pop(i)
       i -= 1
     i+=1
   sortedAllScores = sorted(allScores, key = lambda d: d['score'])
   # les deux derniers éléments de sortedAllScores nous intéressent
   e1 = sortedAllScores[-1]['word']
   e2 = sortedAllScores[-2]['word']
   prob = model.infer(to_input_format(text, e1, e2))
   return [e1, e2, prob[0], prob[1]]
  #  return sortedAllScores

In [163]:
# Test your code with this snippet

sequences = [
             'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', # Easy sentence
             'He was the son of Máel Dúin', # There is only one entity in this sentence, therefore our pipeline should return None
             'Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .', # Ōda is tokenized by the NER tokenizer as a "[UNK]" and it is detected as an entitity. For simplicity, you can discard [UNK] entities as if they were not detected. Here another difficulty arises from the fact that the different tokens in which Iwami Ginzan is decomposed are (wrongly) classified as entities belonging to different classes. In this case, we can consider only the first token "I" as standalone entity, for simplicity.  
             'It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .', # the NER system can not recognise any entity here, therefore the pipeline should return None
             "Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church ." # This sentence is not trivial because the 's has to be managed properly

]

nre = NREPipeline(ner_pipeline, model)
# print(nre(sequences[2]))
for n, sequence in enumerate(sequences):
  out = nre(sequence)
  print("Sentence {0}: {1}".format(n, sequence))
  print("System out: ", out)
  print('------------------------------')

Sentence 0: He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).
System out:  ['Áed Uari', 'Máel Dúin mac Máele Fithrich', 'father', 0.971123456954956]
------------------------------
Sentence 1: He was the son of Máel Dúin
System out:  None
------------------------------
Sentence 2: Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .
System out:  ['World Heritage Site', '##zan', 'heritage designation', 0.9965190887451172]
------------------------------
Sentence 3: It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .
System out:  None
------------------------------
Sentence 4: Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church .
System out:  ['Earleville', "St . Stephen ' s Episcopal Church", 'located in the administrative territorial entit

Par rapport à la sortie que vous avez proposé, nous avons des résultats sensiblement différents car le modèle utilisé n'est pas le même. De plus, nous avons traité les `[UNK]` qui sont des tokens apparaissant quand le mot n'est pas connu du Tokenizer. Pour ce faire, nous supprimons tous les scores qui sont attribués à un texte qui vaut exactement `[UNK]`.

## Application de le pipeline

#### Exercice 6

1. Appliquez la pipeline NRE aux phrases contenues dans le fichier _sentences.txt_, qui ont été extraites à partir de Wikipedia et Wikidata. Donnez un avis qualitatif sur sa performance, les problèmes rencontrés ainsi que des idées pour améliorations des résultats (e.g. entraînement sur des données différentes, modèle différent, etc.).

2. Appliquez la pipeline NRE aux premières 50 phrases contenues dans le fichier _Sentences_AllAgree.txt_ que vous avez utilisé pour le TP BERT, qui parlent d’événements financiers. Remarquez-vous des différences en terme de performances par rapport à la question 2 ? Pourquoi ? Commentez…

**Question 1 :**

In [164]:
with open('../sentences.txt') as f:
    lines = f.readlines()
    for sentence in lines:
      print("---------------------")
      print(sentence)
      print(nre(sentence))

---------------------
The Willard and Josephine Hubbard House was individually listed on the National Register of Historic Places in 2016 .

['Josephine Hubbard', 'Willard', 'spouse', 0.8584034442901611]
---------------------
His station commander , Group Captain Claude Hilton Keith , found a letter among the missing airman 's personal possessions .

None
---------------------
One was Quintus Caecilius Metellus Creticus , who was praetor in 74 BC and consul in 69 BC .

None
---------------------
April 2009 In addition to musical acts , the label recorded beat poets Lawrence Ferlinghetti and Allen Ginsberg and comic Lenny Bruce .

['Lenny Bruce', 'Allen Ginsberg', 'sibling', 0.6458688378334045]
---------------------
Walter Neusel ( November 25 , 1907 – October 3 , 1964 ) was a German heavyweight boxer .

['Walter Neusel', 'German', 'country of citizenship', 0.9982675313949585]
---------------------
Hércules debut defending the national team was a 1938 FIFA World Cup game , played on Jun

On remarque que la performance est élevée lorsqu'il y a peu d'acteurs dans la phrase. Par exemple après traitement de la phrase : `The film was remade in Hindi as " Love Ke Liye Kuch Bhi Karega " and is loosely based on the American film " Ruthless People "`, nous avons en sortie : `['American', 'Hindi', 'original language of film or TV show', 0.8740105032920837]` bien que la phrase soit complexe.

Cependant, pour certaines phrases spécifiques avec beaucoup d'acteurs, le résultat est mitigé. Par exemple : `Euphemia was the youngest child of Władysław Odonic and his wife Jadwiga , disputed daughter of Mestwin I , Duke of Pomerania and Swinisław , daughter of Mieszko III the Old .`. Ici, une relation de parenté entre Władysław Odonic et Swinisław est détectée avec une probabilité de 33%. Pour un cerveau humain, la phrase est intelligible, c'est bien Jadwiga qui est la fille de Swinisław.

Pour améliorer les résultats, on pourrait utiliser un dataset plus spécifiques à ce vocabulaire relativement complexe, on aurait surement de meilleures performances. Cela rejoint les problèmes qui se posent avec un vocabulaire financier assez développé, c.f. question ci-dessous.

**Question 2 :**

In [166]:
# Dans un premier temps, on met les 100 premières phrases du fichier dans un tableau.
# On en profite pour enlever les annotations.
sentences = []
with open('../Sentences_AllAgree.txt', encoding="utf8", errors='ignore') as f:
    lines = f.readlines()
    for i in range(50):
      sentence, emotion = lines[i].split('@', 1)
      sentences.append(sentence)

for sentence in sentences:
  print("---------------------")
  print(sentence)
  print(nre(sentence))

---------------------
According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
['Russia', 'Gran', 'owned by', 0.9612677693367004]
---------------------
For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .
['Componenta', 'EU', 'applies to jurisdiction', 0.8146843910217285]
---------------------
In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .
['EU', 'EU', 'member of', 0.6022771000862122]
---------------------
Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .
['EU', 'EU', 'member of', 0.4233728349208832]
---------------------
Operating profit totalled EUR 21.1 mn , up from EUR 18.6 mn in 2007 , representing 9.7 % of net sales .
['EU

On remarque des différences en terme de performance puisqu'il y a des termes financiers que le tokenizer ne semble pas bien interpréter. Par exemple, pour des devises avec `EUR`, celles-ci sont interprétées comme `EU` soit l'Union Européenne et pose de gros problèmes de compréhension. Les performances sont donc beaucoup moins bonnes, on peut supposer que le tokenizer n'a pas été entrainé sur des phrases si spécifiques.

**Outre le tokenizer, il est également possible de affiner (finetune) les modèles NER et NRE sur un ensemble de données financières. Plus précisément, le modèle de reconnaissance d'entités devrait voir plus d'entités financières (par exemple, des noms de sociétés ou d'institutions financières) dans son apprentissage, et le modèle d'extraction de relations devrait être entraîné avec des relations plus spécifiques, qui ne sont pas présentes dans l'ensemble des 80 relations définies par Wiki80.**